In [ ]:
from platform import python_version
print("Python version", python_version())

Python version 3.10.12


Data uploading: google drive/wget

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#environment setup
%env HOME=/content/drive/MyDrive/
%cd ~/

!mkdir -p ml_com_colab
%cd ml_com_colab
!ls

env: HOME=/content/drive/MyDrive/
/content/drive/MyDrive
/content/drive/MyDrive/ml_com_colab
audio


In [ ]:
!pip install librosa

import os
import tarfile
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

os.chdir('/content/drive/MyDrive/ml_com_colab')


In [ ]:
import zipfile
# Path to the tar files in your Google Drive
test_zip_path = '/content/drive/MyDrive/ML final/train_aug.zip'
train_zip_path = '/content/drive/MyDrive/ML final/test_aug.zip'

def extract_zip_file(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      for file_info in zip_ref.infolist():
            file_path = os.path.join(extract_path, file_info.filename)
            if not os.path.exists(file_path):
                zip_ref.extract(file_info, extract_path)
        #zip_ref.extractall(extract_path)

# Specify the extraction path
extract_train_path = '/content/drive/MyDrive/ML final/unzip_trainnew_aug'
extract_test_path = '/content/drive/MyDrive/ML final/unzip_test_aug'

# Extract files
extract_zip_file(train_zip_path, extract_train_path)
extract_zip_file(test_zip_path, extract_test_path)

In [ ]:
!pip install torchaudio -U

  Using cached torchaudio-2.3.0-cp310-cp310-manylinux1_x86_64.whl (3.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_

In [ ]:
!git clone https://github.com/pytorch/audio.git
os.chdir("audio")
!git checkout 301e2e9
!python setup.py install

fatal: destination path 'audio' already exists and is not an empty directory.
error: pathspec '301e2e9' did not match any file(s) known to git
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
-- Git branch: main
-- Git SHA: ea437b31ce316ea3d66fe73768c0dcb94edb79ad
-- Git tag: None
-- PyTorch dependency: torch
-- Building version 2.2.0a0+ea437b3
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-pa

In [ ]:
!apt-get install sox libsox-dev libsox-fmt-all


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse libsox3 libwavpack1
Suggested packages:
  libaudio2 libsndio6.1
The following NEW packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-dev
  libsox-fmt-all libsox-fmt-alsa libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss
  libsox-fmt-pulse libsox3 libwavpack1 sox
0 upgraded, 17 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,157 kB of archives.
After this operation, 4,262 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libao-common all 1.2.2+20180113-1.1ubuntu3 [6,568 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libao4

In [ ]:
#load audio data

import warnings
warnings.filterwarnings('ignore')

import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import concurrent.futures
import random
import torchaudio
import torch
import time
import re

def numerical_sort(file):
    if file.startswith('.'):
        # Return a large number to push hidden files to the end of the sorted list if they are not filtered out elsewhere
        return float('inf')
    else:
        # Extract the number from the filename using regex
        parts = re.findall(r'\d+', file)
        if parts:
            # Return the first number found in the filename
            return int(parts[0])
        else:
            # Return a large number if no numeric part is found
            return float('inf')

def apply_effects(audio_tensor, sample_rate):
    """Apply audio effects to enhance the audio before spectrogram generation."""
    effects = [
        ["highpass", "300"],  # Highpass filter with cutoff frequency of 300 Hz
        ["lowpass", "3000"],  # Lowpass filter with cutoff frequency of 3000 Hz
        ["gain", "-n", "-5"],  # Normalize volume to -5 dB
        ["reverb", "50"],  # Adding some reverb
        ["echo", "0.8", "0.9", "100", "0.3"]  # Echo effect with decay factors and delays
    ]
    audio_tensor, _ = torchaudio.sox_effects.apply_effects_tensor(
        audio_tensor, sample_rate, effects)
    return audio_tensor

def generate_spectrogram(file_path):
    """Generates and plots a Mel spectrogram for a given audio file."""
    start_time = time.time()

    audio, sample_rate = librosa.load(file_path, sr=None)
    S = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=128, fmax=8000)
    log_S = librosa.power_to_db(S, ref=np.max)
    elapsed_time = time.time() - start_time
    print(f"Generated spectrogram for {file_path} in {elapsed_time:.2f} seconds.")

    return log_S

def process_directory(directory):
    paths = [os.path.join(directory, f) for f in sorted(os.listdir(directory), key=numerical_sort) if not f.startswith('.') and f != float('inf')]
    print("Sorted file paths:")
    spectrograms = []
    for path in paths:
        spectrogram = generate_spectrogram(path)
        spectrograms.append(spectrogram)
    return spectrograms

# Define the path to the directory with extracted files
# Specify the extraction path
extract_test_path = '/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug'
extract_train_path = '/content/drive/MyDrive/ML final/unzip_test_aug/train_aug'

# Process both training and testing directories
train_spectrograms = process_directory(extract_train_path)


流式输出内容被截断，只能显示最后 5000 行内容。
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_test_aug/train_aug/6886.mp3 in 0.04 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_test_aug/train_aug/6887.mp3 in 0.03 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_test_aug/train_aug/6888.mp3 in 0.04 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_test_aug/train_aug/6889.mp3 in 0.03 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_test_aug/train_aug/6890.mp3 in 0.03 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_test_aug/train_aug/6891.mp3 in 0.02 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_test_aug/train_aug/6892.mp3 in 0.03 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_test_aug/train_aug/6893.mp3 in 0.03 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_test_aug/train_aug/6894.mp3 in 0.03 s

In [ ]:
test_spectrograms = process_directory(extract_test_path)

Sorted file paths:
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/0.mp3
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/1.mp3
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/2.mp3
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/3.mp3
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/4.mp3
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/5.mp3
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/6.mp3
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/7.mp3
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/8.mp3
/content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/9.mp3
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/0.mp3 in 0.03 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/1.mp3 in 0.03 seconds.
Generated spectrogram for /content/drive/MyDrive/ML final/unzip_trainnew_aug/test_aug/2.mp3 in 0.02 sec

In [ ]:
#save spectrograms
import numpy as np
import os

def save_spectrograms(spectrograms, directory, filename):
    """Save spectrograms as a numpy file."""
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)
    # Save the numpy array
    np.save(os.path.join(directory, f'{filename}.npy'), np.array(spectrograms))

# Save train and test spectrograms
save_spectrograms(train_spectrograms, '/content/drive/MyDrive/ML final/Saved Spectrograms_echo', 'human_train_spectrograms')
save_spectrograms(test_spectrograms, '/content/drive/MyDrive/ML final/Saved Spectrograms_echo', 'human2_test_spectrograms')
